In [1]:
import os
import sys
import re
import time
from github import Github
from random import randint
from pprint import pprint

In [2]:
from pymongo import MongoClient
pw_file = 'credentials/mongo_pw.txt'
if os.path.exists(pw_file): 
    with open(pw_file, 'r') as f:
        pub_ip, mongo_usr, mongo_usr_pw = f.readline().strip().split(', ')

In [3]:
# connect to ec2 mongo client
client = MongoClient('{0}:27017'.format(pub_ip))

# get reference to  resume_db
db = client['github_db']

# authenticate user for database
db.authenticate(mongo_usr, mongo_usr_pw)

True

In [4]:
for name in sorted(db.collection_names()):
    #db.drop_collection(name)
    print('Documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

Documents in "git_repos_contributors" collection: 3074
Documents in "git_repos_contributors_backup" collection: 3030
Documents in "git_repos_docs" collection: 146833
Documents in "git_repos_docs_backup" collection: 146685
Documents in "git_repos_meta" collection: 3073
Documents in "git_repos_meta_backup" collection: 3029
Documents in "git_repos_subscribers" collection: 3074
Documents in "git_repos_subscribers_backup" collection: 3030
Documents in "git_users_followers" collection: 1398
Documents in "git_users_followers_backup" collection: 1364
Documents in "git_users_following" collection: 1398
Documents in "git_users_following_backup" collection: 1364
Documents in "git_users_meta" collection: 1397
Documents in "git_users_meta_backup" collection: 1363


In [92]:
for name in sorted(db.collection_names()):
    #db.drop_collection(name)
    print('Documents in \"{0}\" collection: {1}'.format(name, db[name].count()))

Documents in "git_repos_contributors" collection: 3074
Documents in "git_repos_contributors_backup" collection: 3030
Documents in "git_repos_docs" collection: 146833
Documents in "git_repos_docs_backup" collection: 146685
Documents in "git_repos_meta" collection: 3073
Documents in "git_repos_meta_backup" collection: 3029
Documents in "git_repos_subscribers" collection: 3074
Documents in "git_repos_subscribers_backup" collection: 3030
Documents in "git_users_followers" collection: 1398
Documents in "git_users_followers_backup" collection: 1364
Documents in "git_users_following" collection: 1398
Documents in "git_users_following_backup" collection: 1364
Documents in "git_users_meta" collection: 1397
Documents in "git_users_meta_backup" collection: 1363


In [6]:
# import github private token
with open('credentials/token.txt', 'r') as infile:
    token = infile.readline().strip()
    user = infile.readline().strip()

git_client = Github(token)

In [7]:
def get_remaining(_type ='core'):
    '''
    _type = 'core' or 'search'
    Return: tuple of remaining rate limit quantity, and time till reset
    '''
    
    rate_limit = git_client.get_rate_limit()
    raw = dict(rate_limit.raw_data)
    
    remaining = int(raw['resources'][_type]['remaining'])
    reset = int(raw['resources'][_type]['reset'])
    
    return (remaining, reset)

In [8]:
print('Resource rate limit remaining: {0}'.format(get_remaining('core')))
print('Search rate limit remaining: {0}'.format(get_remaining('search')))

Resource rate limit remaining: (5000, 1473881417)
Search rate limit remaining: (30, 1473877877)


In [9]:
size = db.command("dbstats")['dataSize']
size = size/1000 #kb
size = size/1000 #mb
print('{0} {1}'.format(int(size), 'mb'))

2129 mb


In [107]:
cursor = db['git_repos_docs_backup'].find({'file_extension':'py'})

pprint(cursor[69000])

{'_id': ObjectId('57d0296b8532183ad7e281cc'),
 'doc_name': '283042__rules.py',
 'file_contents': 'import os import re import inspect from uliweb.utils.common '
                  'import log from uliweb.utils.sorteddict import SortedDict '
                  'from uliweb.utils.date import now import copy class '
                  'ReservedKeyError(Exception):pass __exposes__ = SortedDict() '
                  '__no_need_exposed__ =    __class_methods__ =    '
                  '__app_rules__ =    __url_route_rules__ =    __url_names__ '
                  '=    static_views =    reserved_keys =  , , , , , ,   def '
                  'add_rule(map, url, endpoint=None, **kwargs): from '
                  'werkzeug.routing import Rule kwargs   = endpoint try: '
                  'map.add(Rule(url, **kwargs)) except ValueError as e: '
                  'log.info( % (url, endpoint)) raise def merge_rules(): from '
                  'itertools import chain s =    index =    for v in '
         

In [14]:
cursor = db['git_repos_docs_backup'].find({'file_extension':'py'})

In [15]:
for c in cursor[:5]:
    pprint(c['file_contents'])

('# coding: utf-8 """ The approach taken is explained below. I decided to do '
 'it simply. Initially I was considering parsing the data into some sort of '
 'structure and then generating an appropriate README. I am still considering '
 'doing it - but for now this should work. The only issue I see is that it '
 'only sorts the entries at the lowest level, and that the order of the '
 'top-level contents do not match the order of the actual entries. This could '
 'be extended by having nested blocks, sorting them recursively and flattening '
 'the end structure into a list of lines. Revision 2 maybe ^.^. """ def '
 'main(): # First, we load the current README into memory as an array of lines '
 "with open('README.md', 'r') as read_me_file: read_me = "
 'read_me_file.readlines() # Then we cluster the lines together as blocks # '
 'Each block represents a collection of lines that should be sorted # This was '
 'done by assuming only links ([...](...)) are meant to be sorted # Clustering